In [155]:
%%writefile invoke.yml
# invoke.yml
tasks:
  dev:
    - uv venv
    - uv sync
    - uvicorn api.app.main:app --reload
  test:
    - uv pip install pytest coverage
    - pytest -q
  lint:
    - uv pip install black isort flake8
    - black .
    - isort .
    - flake8


Overwriting invoke.yml


In [156]:
%%writefile .gitignore
.env
dev.env
.devcontainer/.env.runtime

mlruns/
mlflow_db/
mlruns_local/

node_modules/
frontend/node_modules/

archive/
.venv
uv.lock

test_iris.json
#.env.template

# Railway CLI (never commit tokens)
.railway/config.json

archive/

Overwriting .gitignore


In [2]:
%%writefile .env.template
ENV_NAME=react_fastapi_railway
CUDA_TAG=12.8.0
DOCKER_BUILDKIT=1
HOST_JUPYTER_PORT=8890
HOST_TENSORBOARD_PORT=6008
HOST_EXPLAINER_PORT=8050
HOST_STREAMLIT_PORT=8501
HOST_MLFLOW_PORT=5000
HOST_APP_PORT=5100
HOST_BACKEND_DEV_PORT=5002
MLFLOW_TRACKING_URI=http://mlflow:5000
MLFLOW_VERSION=2.12.2
PYTHON_VER=3.10
JAX_PLATFORM_NAME=gpu
XLA_PYTHON_CLIENT_PREALLOCATE=true
XLA_PYTHON_CLIENT_ALLOCATOR=platform
XLA_PYTHON_CLIENT_MEM_FRACTION=0.95
XLA_FLAGS=--xla_force_host_platform_device_count=1
JAX_DISABLE_JIT=false
JAX_ENABLE_X64=false
TF_FORCE_GPU_ALLOW_GROWTH=false
JAX_PREALLOCATION_SIZE_LIMIT_BYTES=8589934592

RAILWAY_TOKEN=
RAILWAY_VITE_API_URL=https://fastapi-production-1d13.up.railway.app
VITE_API_URL=http://127.0.0.1:8000
REACT_APP_API_URL=https://react-frontend-production-2805.up.railway.app

SECRET_KEY=change-me-in-prod
USERNAME_KEY=alice
USER_PASSWORD=supersecretvalue
DATABASE_URL=sqlite+aiosqlite:///./app.db
FRONTEND_URL=http://localhost:5173


Overwriting .env.template


In [158]:
%%writefile pyproject.toml
[project]
name = "react_fastapi_railway"
version = "0.1.0"
description = "Pytorch and Jax GPU docker container"
authors = [
  { name = "Geoffrey Hadfield" },
]
license = "MIT"
readme = "README.md"

# ─── Restrict to Python 3.10–3.12 ──────────────────────────────
requires-python = ">=3.10,<3.13"

dependencies = [
  # Core web framework
  "fastapi>=0.104.0",
  "uvicorn[standard]>=0.24.0",
  "python-dotenv>=1.0.0",
  
  # Settings and validation
  "pydantic>=2.0.0",
  "pydantic-settings>=2.0.0",

  # HTTP client and multipart parsing
  "httpx>=0.24.0",
  "python-multipart>=0.0.6",

  # Data & ML basics
  "numpy>=1.24.0",
  "pandas>=2.1.0",
  "scikit-learn>=1.3.0",
  "mlflow>=2.8.0",

  # (Your existing extras—keep if you still need them)
  "matplotlib>=3.4.0",
  "mlflow-skinny>=2.12.2",
  "pymc>=5.0.0",
  "arviz>=0.14.0",
  "statsmodels>=0.13.0",
  "jupyterlab>=3.0.0",
  "seaborn>=0.11.0",
  "tabulate>=0.9.0",
  "shap>=0.40.0",
  "xgboost>=1.5.0",
  "lightgbm>=3.3.0",
  "catboost>=1.2.8,<1.3.0",
  "scipy>=1.7.0",
  "shapash[report]>=2.3.0",
  "shapiq>=0.1.0",
  "explainerdashboard==0.5.1",
  "ipywidgets>=8.0.0",
  "nutpie>=0.7.1",
  "numpyro>=0.18.0,<1.0.0",
  "jax==0.6.0",
  "jaxlib==0.6.0",
  "pytensor>=2.18.3",
  "aesara>=2.9.4",
  "tqdm>=4.67.0",
  "pyarrow>=12.0.0",
  "optuna>=3.0.0",
  "optuna-integration[mlflow]>=0.2.0",
  "omegaconf>=2.3.0,<2.4.0",
  "hydra-core>=1.3.2,<1.4.0",
  "aiosqlite>=0.19.0", 
  "python-jose[cryptography]>=3.3.0",
  "passlib[bcrypt]>=1.7.4",
  "bcrypt==4.0.1",  # Pin bcrypt version to resolve warning
]

[project.optional-dependencies]
dev = [
  "pytest>=7.0.0",
  "black>=23.0.0",
  "isort>=5.0.0",
  "flake8>=5.0.0",
  "mypy>=1.0.0",
  "invoke>=2.2",
]

cuda = [
  "cupy-cuda12x>=12.0.0",
]

[tool.pytensor]
device    = "cuda"
floatX    = "float32"
allow_gc  = true
optimizer = "fast_run"


Overwriting pyproject.toml


In [159]:
%%writefile api/pyproject.toml
[project]
name = "api"
version = "1.0.0"
description = "FastAPI backend with React frontend"
requires-python = ">=3.8"
dependencies = [
    "fastapi>=0.104.0",
    "uvicorn>=0.24.0",
    "sqlalchemy>=2.0.23",
    "aiosqlite>=0.19.0",
    "python-jose[cryptography]>=3.3.0",
    "passlib[bcrypt]>=1.7.4",
    "python-multipart>=0.0.6",
    "pydantic>=2.4.2",
    "bcrypt==4.0.1"  # Pin bcrypt version to resolve warning
]

[build-system]
requires = ["hatchling"]
build-backend = "hatchling.build"

[tool.hatch.build.targets.wheel]
packages = ["app"]



Overwriting api/pyproject.toml


In [160]:
%%writefile api/railway.json
{
  "$schema": "https://railway.app/railway.schema.json",
  "build": {
    "builder": "NIXPACKS"
  },
  "deploy": {
    "startCommand": "uvicorn app.main:app --host 0.0.0.0 --port $PORT",
    "restartPolicyType": "ON_FAILURE",
    "restartPolicyMaxRetries": 10,
    "healthcheckPath": "/api/health",
    "healthcheckTimeout": 300
  }
} 

Overwriting api/railway.json


In [161]:
%%writefile api/scripts/seed_user.py
from importlib import util
if util.find_spec("passlib") is None:  # noqa: E402
    raise RuntimeError("Run `npm run install:all` before seeding; passlib is missing")

from passlib.context import CryptContext  # noqa: E402
from sqlalchemy.ext.asyncio import create_async_engine, async_sessionmaker
from sqlalchemy import select
import sys
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Add the parent directory to sys.path
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(__file__))))
from api.app.models import Base, User

# Get credentials from environment with fallbacks
USERNAME = os.getenv("USERNAME_KEY", "alice")
PASSWORD = os.getenv("USER_PASSWORD", "secret")

engine = create_async_engine("sqlite+aiosqlite:///./app.db", future=True)
async_session = async_sessionmaker(engine, expire_on_commit=False)
pwd = CryptContext(schemes=["bcrypt"], deprecated="auto")

async def main():
    async with engine.begin() as conn:
        await conn.run_sync(Base.metadata.create_all)

    async with async_session() as db:
        # Check if user exists
        result = await db.execute(select(User).filter_by(username=USERNAME))
        user = result.scalar_one_or_none()

        hashed_password = pwd.hash(PASSWORD)
        
        if user is None:
            # Create new user if doesn't exist
            u = User(username=USERNAME, hashed_password=hashed_password)
            db.add(u)
            action = "Created"
        else:
            # Update existing user's password
            user.hashed_password = hashed_password
            action = "Updated"
            
        await db.commit()
        print(f"{action} user '{USERNAME}' with password '{PASSWORD}'")

if __name__ == "__main__":
    import asyncio
    asyncio.run(main()) 



Overwriting api/scripts/seed_user.py


In [162]:
%%writefile api/app/__init__.py
# FastAPI backend package 

Overwriting api/app/__init__.py


In [163]:
%%writefile api/app/crud.py
from sqlalchemy import select
from sqlalchemy.ext.asyncio import AsyncSession
from .models import User

async def get_user_by_username(db: AsyncSession, username: str):
    stmt = select(User).where(User.username == username)
    res = await db.execute(stmt)
    return res.scalar_one_or_none() 

Overwriting api/app/crud.py


In [164]:
%%writefile api/app/models.py
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import declarative_base

Base = declarative_base()

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, unique=True, index=True)
    hashed_password = Column(String) 

Overwriting api/app/models.py


In [165]:
%%writefile api/app/db.py
# api/app/db.py
from contextlib import asynccontextmanager
import os
from sqlalchemy.ext.asyncio import AsyncSession, create_async_engine, async_sessionmaker
from .models import Base

DATABASE_URL = os.getenv("DATABASE_URL", "sqlite+aiosqlite:///./app.db")

engine = create_async_engine(DATABASE_URL, echo=False, future=True)
AsyncSessionLocal = async_sessionmaker(engine, expire_on_commit=False)

@asynccontextmanager
async def lifespan(app):
    """Open & dispose engine on startup/shutdown."""
    async with engine.begin() as conn:
        await conn.run_sync(Base.metadata.create_all)
    yield
    await engine.dispose()

async def get_db() -> AsyncSession:
    """Yield a new DB session for each request."""
    async with AsyncSessionLocal() as session:
        yield session



Overwriting api/app/db.py


In [166]:
%%writefile api/app/security.py
# api/app/security.py
from datetime import datetime, timedelta
from typing import Optional
import os
from fastapi import Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from passlib.context import CryptContext
from jose import JWTError, jwt
from pydantic import BaseModel

# api/app/security.py  (replace the top-of-file guard)
SECRET_KEY = os.getenv("SECRET_KEY")
if not SECRET_KEY:                      # ← treat "" exactly like missing
    raise RuntimeError(
        "SECRET_KEY env var is required and must be non-empty. "
        "Add one to .env or set it in production secrets store."
    )

ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="/api/token")

class TokenData(BaseModel):
    username: Optional[str] = None

def verify_password(plain: str, hashed: str) -> bool:
    return pwd_context.verify(plain, hashed)

def get_password_hash(password: str) -> str:
    return pwd_context.hash(password)

def create_access_token(subject: str) -> str:
    expire = datetime.utcnow() + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    to_encode = {"sub": subject, "exp": expire}
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

async def get_current_user(token: str = Depends(oauth2_scheme)) -> str:
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username: str = payload.get("sub")
        if not username:
            raise HTTPException(status_code=status.HTTP_401_UNAUTHORIZED)
        return username
    except JWTError:
        raise HTTPException(status_code=status.HTTP_401_UNAUTHORIZED)


Overwriting api/app/security.py


In [167]:
%%writefile api/app/main.py
import logging
import os
from fastapi import FastAPI, Request, Depends, BackgroundTasks, status, HTTPException
from fastapi.security import OAuth2PasswordRequestForm
from fastapi.middleware.cors import CORSMiddleware
from sqlalchemy.ext.asyncio import AsyncSession
from pydantic import BaseModel
import time

from .db import lifespan, get_db
from .security import create_access_token, get_current_user, verify_password
from .crud import get_user_by_username

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Pydantic models
class Payload(BaseModel):
    count: int

class PredictionRequest(BaseModel):
    data: Payload

class PredictionResponse(BaseModel):
    prediction: str
    confidence: float
    input_received: Payload  # Echo back the input for verification

app = FastAPI(
    title="FastAPI + React App",
    version="1.0.0",
    docs_url="/api/docs",
    redoc_url="/api/redoc",
    openapi_url="/api/openapi.json",
    swagger_ui_parameters={"persistAuthorization": True},
    lifespan=lifespan,  # register startup/shutdown events
)

# Configure CORS
origins = [
    "http://localhost:5173",  # Vite dev server
    "http://localhost:3000",  # Alternative dev port
]

# Add production frontend URL if available
frontend_url = os.getenv("FRONTEND_URL")
if frontend_url:
    origins.append(frontend_url)

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    """Measure request time and add X-Process-Time header."""
    start = time.perf_counter()
    response = await call_next(request)
    elapsed = time.perf_counter() - start
    response.headers["X-Process-Time"] = f"{elapsed:.4f}"
    return response

@app.get("/api/health")
async def health_check():
    """Unprotected health check endpoint."""
    return {"status": "healthy"}

@app.get("/api/hello")
async def hello(current_user: str = Depends(get_current_user)):
    """Protected hello endpoint that returns a greeting."""
    return {"message": f"Hello {current_user}!"}

@app.post("/api/token")
async def login(
    form_data: OAuth2PasswordRequestForm = Depends(),
    db: AsyncSession = Depends(get_db),
):
    """Authenticate user and issue JWT."""
    user = await get_user_by_username(db, form_data.username)
    if not user or not verify_password(form_data.password, user.hashed_password):
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Invalid credentials"
        )
    token = create_access_token(subject=user.username)
    return {"access_token": token, "token_type": "bearer"}

@app.post(
    "/api/predict",
    response_model=PredictionResponse,
    status_code=status.HTTP_200_OK
)
async def predict(
    request: PredictionRequest,
    background_tasks: BackgroundTasks,
    db: AsyncSession = Depends(get_db),
    current_user: str = Depends(get_current_user),
):
    """Protected prediction endpoint with DB session & background auditing.
    
    Example request:
        {
            "data": {
                "count": 42
            }
        }
    """
    logger.info(f"User {current_user} called /predict with count={request.data.count}")

    # Mock prediction - replace with your actual ML model
    result = {
        "prediction": "sample",
        "confidence": 0.95,
        "input_received": request.data
    }

    # Background task for audit logging
    background_tasks.add_task(
        logger.info,
        f"[audit] user={current_user} input={request.data} output={result}"
    )

    return PredictionResponse(**result) 





Overwriting api/app/main.py
